Experiment with OpenCV for finding the pool table edges within an image

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
import math
from Utils import *
import copy

In [ ]:
# Do the calibration
dirCalImages = "AmcrestCamera/Checkerboards"

if not CalibrationExists():
    print("Calibrating for camera lens distortion")
    GenerateCalibration(dirCalImages, False)
else:
    print("skipping calibration, using cached calibration")
    
files, dir = FindImageFilesAndDir(subdir=dirCalImages)
filenameTest = files[0]

# Hack-in some ROI tweeking (xywh)
roiNew = (356, 293, 1422+100, 3228-250)
SetCalibrationROI(roiNew)

imageTest = cv.imread(filenameTest)
imageTestClipped, imageTestUnclipped = CalibrateImage(imageTest)
ImgShow([imageTestClipped, imageTestUnclipped], 210)

In [ ]:
# Load the test image
dirImages = "AmcrestCamera/Session3"
files, dir = FindImageFilesAndDir(subdir=dirImages)
filename = files[0]
filenameEmpty = "{0}/empty43.jpg".format(dir)

print(f'Loading and empty table file {filenameEmpty}')
imageEmpty = cv.imread(filenameEmpty)
imageEmptyClipped, imageEmptyUnclipped = CalibrateImage(imageEmpty)

print(f'Loading and calibrating file {filename}')
imageOriginal = cv.imread(filename)
imageOriginalClipped, imageOriginalUnclipped = CalibrateImage(imageOriginal)

ImgShow([imageOriginalUnclipped, imageOriginalClipped, imageEmptyUnclipped, imageEmptyClipped], 275)

imageOriginal = imageOriginalClipped
imageEmpty = imageEmptyClipped

In [ ]:
#Find the edges using flood, start with a blur
blur = int(85)
imageBlur = cv.GaussianBlur(imageOriginal, (blur,blur), cv.BORDER_DEFAULT)

imageFeltFlood = imageBlur.copy()
cols = imageFeltFlood.shape[1]
rows = imageFeltFlood.shape[0]
seed = (int(cols / 2), int(rows / 2)) # a point in the middle

imageFeltFloodMaskBrdr = np.zeros((rows+2,cols+2,1), np.uint8)
tolerance = 34.0
diff = (tolerance,tolerance,tolerance)
flags = 8 | cv.FLOODFILL_FIXED_RANGE | 255 << 8
res = cv.floodFill(imageFeltFlood, imageFeltFloodMaskBrdr, seed, (0,0,255),diff,diff, flags)

imageFeltFloodMask = imageFeltFloodMaskBrdr[1:rows+1, 1:cols+1]
print("imageFeltFlood shape         = {0}".format(imageFeltFlood.shape))
print("imageFeltFloodMaskBrdr shape = {0}".format(imageFeltFloodMaskBrdr.shape))
print("imageFeltFloodMask shape    = {0}".format(imageFeltFloodMask.shape))
ImgShow([imageBlur, imageFeltFlood, imageFeltFloodMask], 220)

In [ ]:
# HoughLinesP for detection

#filters image bilaterally and displays it
bilatImg = cv.bilateralFilter(imageOriginal, d=5, sigmaColor=175, sigmaSpace=75)

#finds edges of bilaterally filtered image and displays it
imageCanny = cv.Canny(bilatImg, 100, 100)

img = imageCanny

cols = imageOriginal.shape[1]
rows = imageOriginal.shape[0]
shortSide = min(cols, rows)
minLineLen = shortSide / 4

rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 135  # minimum number of votes (intersections in Hough grid cell)
min_line_length = minLineLen  # minimum number of pixels making up a line
max_line_gap = 30  # maximum gap in pixels between connectable line segments

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
tableLines = cv.HoughLinesP(img, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)
print("tableLines count = " + str(len(tableLines)))

# For visualization, draw the lines onto a copy of the image
imageDrawnLines = np.zeros(img.shape[:3],np.uint8)  # creating a blank to draw lines on

pts = []
for line in tableLines:
    for x1,y1,x2,y2 in line:
        pt1 = [x1,y1]
        pt2 = [x2,y2]
        cv.line(imageDrawnLines,pt1,pt2,(255,0,0),3)
        pts.append(pt1)
        pts.append(pt2)
        
# creating convex hull object for each contour
hull = cv.convexHull(np.array(pts), False)
#print("hull=", hull)

# Convert hull to points list
ptsHullRaw = []
for cell in hull:
    ptsHullRaw.append(cell[0])
ptsHull = np.array(ptsHullRaw, dtype=np.int32)
ptsHull.reshape((-1,1,2))

imageConvexMask = np.zeros(img.shape[:3],np.uint8) 
cv.fillConvexPoly(imageConvexMask, np.array(ptsHull, dtype=np.int32), (255))

# Subtract out the felt, mask out the internal felt by using a bitwise-and
imageFeltFloodMaskInv = cv.bitwise_not(imageFeltFloodMask)
imageWoodRailsMask = cv.bitwise_and(imageConvexMask, imageConvexMask, mask=imageFeltFloodMaskInv)
ret,imageWoodRailsMask = cv.threshold(imageWoodRailsMask,127,255,cv.THRESH_BINARY)

# Apply the mask to the original image using bitwise-and
imageWoodRailsMaskRGB = cv.cvtColor(imageWoodRailsMask, cv.COLOR_GRAY2BGR)
ret,imageWoodRailsMaskRGB = cv.threshold(imageWoodRailsMaskRGB,127,255,cv.THRESH_BINARY)
imageWoodRails = cv.bitwise_and(imageOriginal, imageWoodRailsMaskRGB)

ImgShow([img, imageDrawnLines, imageConvexMask, imageWoodRailsMask, imageWoodRails], 220)

In [ ]:
# Hough Circles. 
imageInput = imageWoodRails
print("imageInput.shape = {0}".format(imageInput.shape))

# Limit by size range and distance
cols = imageInput.shape[1]
rows = imageInput.shape[0]
imageGray = cv.cvtColor(imageInput, cv.COLOR_BGR2GRAY)
imageGray = cv.GaussianBlur(imageGray, (7,7), 2,2)
hcircles = cv.HoughCircles(imageGray, cv.HOUGH_GRADIENT, minDist=60, dp=1,
                               param1=100, param2=10,
                               minRadius=2, maxRadius=15)

# Create an array of points
markerPoints = []
imageCircles = imageInput.copy()
if hcircles is not None:
    hcircles = np.uint16(np.around(hcircles))
    for i in hcircles[0, :]: 
        pt = [i[0], i[1]]
        radius = i[2]
    
        # Filter out points ouside of image
        if pt[0] < 0 or pt[1] < 0:
            continue
        if pt[0] > cols-1 or pt[1] > rows-1:
            continue
    
        markerPoints.append(pt)
        cv.circle(imageCircles, pt, radius, Color(), 7)
        
print("markerPoints count = {0}".format(len(markerPoints)))
    
ImgShow([imageInput, imageCircles], 320)

In [ ]:
# Filter out circles that don't match the color we want
# We will assume that the circle centers should be whiteish

imageIn = imageWoodRails

# Find the range
clrWhite = (240, 240, 240)
fuzz = 45

imageCirclesWhite = imageIn.copy()
imageCirclesWhiteBlack = np.zeros(imageIn.shape[:3],np.uint8)

clrMin, clrMax = ColorRange(clrWhite, fuzz)
clrMin = (clrMin[0]-1,clrMin[1]-1,clrMin[2]-1,)
clrMax = (clrMax[0]+1,clrMax[1]+1,clrMax[2]+1,)
print("color range = {0} - {1}".format(clrMin, clrMax))
circlesWhite = []
circRadius = 19
for pt in markerPoints:    
    clr = imageIn[pt[1],pt[0]]
    if (clr > clrMin).all() and (clr < clrMax).all():
        circlesWhite.append(pt)
        clr = Color()
        cv.circle(imageCirclesWhite, pt, circRadius, clr, 6)
        cv.circle(imageCirclesWhiteBlack, pt, circRadius, clr, 6)
        #print("pt={0} clr={1}".format(pt, clr))
print("Marker Prunage: markerPoints count={0}, circlesWhite count={1}".format(len(markerPoints), len(circlesWhite)))

ImgShow([imageCircles, imageCirclesWhite, imageCirclesWhiteBlack], 220)

In [ ]:
# Group the markers into lines

# Find the top and bottom points by scanning all points
ymin = 10E10
ymax = 0
for pt in circlesWhite:
    if pt[1] < ymin:
        ymin = pt[1]
    if pt[1] > ymax:
        ymax = pt[1]

# Sort the markers
markersL = []
markersR = []
markersT = []
markersB = []

yFuzz = int(imageOriginal.shape[0] / 20)    # search range
xmid = int(imageOriginal.shape[1] / 2)
for pt in circlesWhite:
    if pt[1] < ymin + yFuzz:
        markersT.append(pt)
    else:
        if pt[1] > ymax - yFuzz:
            markersB.append(pt)
        else:
            if pt[0] < xmid:
                markersL.append(pt)
            else:
                markersR.append(pt)

# Sort by position, helpful for imputing new phantom markers later on
markersL.sort(key = lambda circ: circ[1])
markersR.sort(key = lambda circ: circ[1])
markersT.sort(key = lambda circ: circ[0])
markersB.sort(key = lambda circ: circ[0])

# Another view into it
markersTLBR = [markersT, markersL, markersB, markersR]

print("markersL", len(markersL), markersL)
print("markersR", len(markersR), markersR)
print("markersT", len(markersT), markersT)
print("markersB", len(markersB), markersB)

In [ ]:
# Show on an image just for sanity
imageMarkerLines = imageOriginal.copy()

def DrawMarkers(img, mks, clrLine=(0,0,0), clrMark=(0,0,0)):
    pt1 = mks[0]
    pt2 = mks[len(mks) - 1]
    
    if clrLine == (0,0,0):
        clrMark = Color()
        
    if clrMark == (0,0,0):
        clrMark = Color()
    
    # int
    pt1 = (int(pt1[0]), int(pt1[1]))
    pt2 = (int(pt2[0]), int(pt2[1]))
    
    cv.line(img, pt1, pt2, clrLine, 4)
    for pt in mks:
        pt = (int(pt[0]), int(pt[1]))
        #print(pt)
        cv.circle(img, pt, 25, clrMark, 4)
        
DrawMarkers(imageMarkerLines, markersL)
DrawMarkers(imageMarkerLines, markersR)
DrawMarkers(imageMarkerLines, markersT)
DrawMarkers(imageMarkerLines, markersB)
            
ImgShow([imageMarkerLines], 210)

In [ ]:
# Find points on the lines
# There will likely be some points that don't belong, use linear regression line fitting
# to filter those out

def GetFit(pts):
    #print("GetFit")
    data_x = pts[:,0]
    data_y = pts[:,1]
    ret = np.polyfit(data_x, data_y, 1, full=True)
    coeffs, resid, rank, singular_values, rcond = ret
    if 2 == len(pts):
        resid = [0.0]   # Special case, we always get a fit with just two points
        
    # Convert the residual to an average distance. It is currently sum of the square of the distances.
    r = resid[0]
    dAve = math.sqrt(r / len(data_x))
    #print(f'r={r} dAve={dAve}')
    return coeffs, dAve

# Try until we find a great fit or fail (RECURSIVE!)    
def FindLinePoints(pts, rangeSize, maxAveDist):    
    # Convert to numpy array if needed
    if 'numpy.ndarray' != type(pts):
        pts = np.array(pts, dtype=np.int32)
    cnt = len(pts)
    
    
    minPoints = rangeSize[0]
    maxPoints = rangeSize[1]
    if cnt < minPoints:
        return [], maxAveDist
    
    # Prune the top-level sets down to the max size, we won't search
    # point sets larger than the max size
    subsets = [pts]
    extraPts = len(pts) - maxPoints
    while extraPts > 0:
        # Remove a point in all possible ways
        splitsets = []
        for ptsOne in subsets:
            for i in range(0, len(ptsOne)):
                ptsMinusOne = np.delete(ptsOne, i, axis=0)
                splitsets.append(ptsMinusOne)
        extraPts -= 1
        subsets = splitsets    
        
    # The order of the search matters (we want the largest set allowed that is a line),
    # so we delay recursion until we have processed all top-level
    # sets. 
    distances = []
    bestDistVal = 10E10 # big enough for anything
    bestDistIndex = -1
    for idx, ptsSubSet in enumerate(subsets):
        coeffs, aveDist = GetFit(ptsSubSet)
        #print(f' i={i} aveDist={aveDist}')
        distances.append(aveDist)
        if aveDist < bestDistVal:
            # We found a new best fit, remember it
            bestDistVal = aveDist
            bestDistIndex = idx
            
    # Can we stop?
    if bestDistVal < maxAveDist:
        #print(" found")
        return subsets[bestDistIndex], bestDistVal  # Yes, we have a nice fit that's good enough to keep
    
    # We don't have a nice fit, get recursive
    maxPoints -= 1
    if maxPoints < minPoints:
        return [], maxAveDist # we are at the end of our search
    
    distances = []
    bestDistVal = 10E10 # big enough for anything
    bestDistIndex = -1
    for subset in subsets:
        ptsFit, aveDist = FindLinePoints(subset, (minPoints, maxPoints), maxAveDist)
        distances.append(aveDist)
        if aveDist < bestDistVal:
            # We found a new best fit, remember it
            bestDistVal = aveDist
            bestDistIndex = idx
            
    # Do we have something we are pleased with?
    if bestDistVal < maxAveDist:
        #print(" found")
        return subsets[bestDistIndex], bestDistVal  # Yes, we have a nice fit that's good enough to keep
        
    # Did not find it
    return [], maxAveDist


def ProcessLine(name, pts, range, maxAveDist, process=True):
    if not process:
        return pts
    ptsFit, aveDist = FindLinePoints(pts, range, maxAveDist)
    cntB = len(pts)
    cntA = len(ptsFit)
    print(f'name={name} range={range} maxAveDist={maxAveDist} len: {cntB}->{cntA}')
    return ptsFit



maxAveDist = 375.0
markersR2 = ProcessLine("markersR2", markersR, range=(6,6), maxAveDist=maxAveDist)
markersL2 = ProcessLine("markersL2", markersL, range=(6,6), maxAveDist=maxAveDist)
markersT2 = ProcessLine("markersT2", markersT, range=(2,3), maxAveDist=maxAveDist)
markersB2 = ProcessLine("markersB2", markersB, range=(2,3), maxAveDist=maxAveDist)
    

imageMarkerLinesPruned = imageOriginal.copy()
DrawMarkers(imageMarkerLinesPruned, markersR2)
DrawMarkers(imageMarkerLinesPruned, markersL2)
DrawMarkers(imageMarkerLinesPruned, markersT2)
DrawMarkers(imageMarkerLinesPruned, markersB2)

ImgShow([imageMarkerLinesPruned], 210)


In [ ]:
# Find the corner points of intersection

def FindLine(mks):
    pt1 = mks[0]
    pt2 = mks[len(mks) - 1]
    x1 = pt1[0]
    y1 = pt1[1]
    x2 = pt2[0]
    y2 = pt2[1]
    rise = float(float(y2) - float(y1))
    run  = float(float(x2) - float(x1))
    if run != 0.0:
        m = rise/run
    else:
        m = float(10E10)
    b = float(y1 - m * x1)
    return (m,b)


def FindIntersection(mksA, mksB):
    lineA = FindLine(mksA)
    lineB = FindLine(mksB)
    a = lineA[0]
    c = lineA[1]
    b = lineB[0]
    d = lineB[1]
    
    x = (d-c)/(a-b)
    y = a*x+c
    return (round(x),round(y))


ptTL = FindIntersection(markersL2, markersT2)
ptTR = FindIntersection(markersT2, markersR2)
ptBL = FindIntersection(markersL2, markersB2)
ptBR = FindIntersection(markersR2, markersB2)

imageCorners = imageMarkerLinesPruned.copy()
print(ptTL)
dia = 175
clrCorner = (0,0,255)
fill = 10
cv.circle(imageCorners, ptTL, dia, clrCorner, fill)
cv.circle(imageCorners, ptTR, dia, clrCorner, fill)
print(ptBL)
cv.circle(imageCorners, ptBL, dia, clrCorner, fill)
cv.circle(imageCorners, ptBR, dia, clrCorner, fill)

ImgShow([imageCorners], 200)

In [ ]:
# Perspective Transform
rows = imageOriginal.shape[0]
cols = imageOriginal.shape[1]

# Convert a scalar point to an array
def ToPtArray(ptIn):
    ptsOut = []
    x,y = ptIn
    ptsOut = [x,y]
    return ptsOut

# The transformed space should be in dimensions that have the 
# correct aspect ratio for a pool table (2:1)
if rows > cols:
    tgtDims = (cols, cols*2)
else:
    tgtDims = (rows*2, rows)
print("Target Dims = {0}".format(tgtDims))

# Build the array of src and dst points
ptsTransformSrc = np.array( [
    ToPtArray(ptTL),
    ToPtArray(ptTR), 
    ToPtArray(ptBR),
    ToPtArray(ptBL) ]
         ).astype(np.float32)

ptsTransformDst = np.array( [    
    [0,0],
    [tgtDims[0]-1, 0],
    [tgtDims[0]-1, tgtDims[1]-1],
    [0, tgtDims[1]-1]] 
         ).astype(np.float32)

# Calculate the transform matrix
matTxToFlat = cv.getPerspectiveTransform(ptsTransformSrc, ptsTransformDst)
matTxFromFlat = cv.getPerspectiveTransform(ptsTransformDst, ptsTransformSrc)


# Apply the warp transform
imageWarp = cv.warpPerspective(imageCorners, matTxToFlat, tgtDims)

print("Perspective Transform")
ImgShow([imageCorners, imageWarp],240)

In [ ]:
# Transform functions

def NPA(inp):
    # Convert to float array
    rows = []
    for row in inp:
        rowArray = []
        for c in row:
            rowArray.append(float(c))
        rows.append(rowArray)
        
    # Build a numpy array
    return np.array(rows).astype(np.float32)

def TxPoint(matTx, pt):
    c3 = matTx[2,0]*pt[0] + matTx[2,1]*pt[1] + matTx[2,2]
    m = [(matTx[0,0]*pt[0] + matTx[0,1]*pt[1] + matTx[0,2])/c3,
         (matTx[1,0]*pt[0] + matTx[1,1]*pt[1] + matTx[1,2])/c3]
    return m

def TxPoints(matTx, pts):
    out = []
    for pt in pts:
        a = TxPoint(matTx, pt)
        out.append(a)
    return out

matTx = NPA([[2,0.5,-100],
            [0,2,0],
            [0,0.005,1]])

ptTx = TxPoint(matTx, [100.0, 100.0])
print("ptTx",ptTx)
X = TxPoints(matTx, 
                [[100.0, 100.0],
                 [100.0, 100.0],
                 [100.0, 100.0],
                 [100.0, 100.0]])
print("X",X)
# Should be [100.00000074505806, 133.33333432674408]]

# We can go both ways, test to be sure
pt =  [500,1000]
print("pt", [500,1000])
ptTx = TxPoint(matTxToFlat, pt)
ptOrig = TxPoint(matTxFromFlat, ptTx)
print("ptFlat",ptTx)
print("ptOrig",ptOrig)

In [ ]:
# Add the missing middle markers
# Now that we can work in "flat space", it's easier to impute
# the 'missing' markers over the side pockets and end plaque
 
def FindMissingMiddle(mks):
    cnt = len(mks)
    if 0 != (cnt % 2):
        return mks # Only add if even count
    
    # Transform to flat space
    mks = TxPoints(matTxToFlat, mks)
    
    # Find the middle points
    idxL = int(len(mks)/2)-1
    idxR = idxL+1
    
    # Interpolate a point between the two
    ptA = mks[idxL]
    ptB = mks[idxR]
    dx = float(ptB[0]) - float(ptA[0])
    dy = float(ptB[1]) - float(ptA[1])
    ptMiddle = (int(ptA[0] + dx/2), int(ptA[1] + dy/2))
    mks.insert(idxR, ptMiddle)
    
    # Transform back to native space
    mks = TxPoints(matTxFromFlat, mks)
    return mks

markersL3 = FindMissingMiddle(markersL2)
markersR3 = FindMissingMiddle(markersR2)
markersT3 = FindMissingMiddle(markersT2)
markersB3 = FindMissingMiddle(markersB2)

imageMarkerMissing = imageOriginal.copy()
DrawMarkers(imageMarkerMissing, markersL3)
DrawMarkers(imageMarkerMissing, markersR3)
DrawMarkers(imageMarkerMissing, markersT3)
DrawMarkers(imageMarkerMissing, markersB3)

ImgShow(imageMarkerMissing, 180)

In [ ]:
# Find and add the missing 'phantom' markers

def Distance(pt1, pt2):
    dx = float(pt2[0])-float(pt1[0])
    dy = float(pt2[1])-float(pt1[1])
    return math.sqrt(dx*dx + dy*dy)

def AverageDistance(mks):
    sum = 0.0
    xsum = 0.0
    ysum = 0.0
    for i in range(1, len(mks)):
        d = Distance(mks[i], mks[i-1])
        sum += d
        xsum += float(mks[i][0]) - float(mks[i-1][0])
        ysum += float(mks[i][1]) - float(mks[i-1][1])
    ave = sum / (float(len(mks)-1))
    dx = xsum / (float(len(mks)-1))
    dy = ysum / (float(len(mks)-1))
    return dx, dy

def AddPhantomPoints(mks):
    # Transform the marker positions into flat space
    mks = TxPoints(matTxToFlat, mks)
    dx, dy = AverageDistance(mks)
    last = len(mks) - 1
    pt0 = [mks[0][0] - dx, mks[0][1] - dy]
    ptN = [mks[last][0] + dx, mks[last][1] + dy]
    mks.insert(0, pt0)
    mks.append(ptN)
    
    # Back to native space
    mks = TxPoints(matTxFromFlat, mks)
    return mks
    
markersL4 = AddPhantomPoints(markersL3)
markersR4 = AddPhantomPoints(markersR3)
markersT4 = AddPhantomPoints(markersT3)
markersB4 = AddPhantomPoints(markersB3)


# Add the start/end 'phantom' markers

# The new 'phantom' markers are at the 


imageMarkerMissing = imageOriginal.copy()
clrLine = (0,255,255)
DrawMarkers(imageMarkerMissing, markersL4, clrLine=clrLine)
DrawMarkers(imageMarkerMissing, markersR4, clrLine=clrLine)
DrawMarkers(imageMarkerMissing, markersT4, clrLine=clrLine)
DrawMarkers(imageMarkerMissing, markersB4, clrLine=clrLine)

ImgShow(imageMarkerMissing, 200)


In [ ]:
# Use the phantom markers for new side lines
# This is the 'playing area'

# Make a copy for prettier math
mksL = markersL4
mksR = markersR4
mksT = markersT4
mksB = markersB4

mksPaT = [mksL[0], mksR[0]]
mksPaB = [mksL[len(mksL)-1], mksR[len(mksR)-1]]
mksPaL = [mksT[0], mksB[0]]
mksPaR = [mksT[len(mksT)-1], mksB[len(mksB)-1]]

imageSides = imageOriginal.copy()

# Draw the original marker lines
DrawMarkers(imageSides, mksL)
DrawMarkers(imageSides, mksR)
DrawMarkers(imageSides, mksT)
DrawMarkers(imageSides, mksB)

# Draw the new play area lines in a different color
clrLine = (255,255,0)
clrMark= (255,0,0)
DrawMarkers(imageSides, mksPaT, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageSides, mksPaB, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageSides, mksPaL, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageSides, mksPaR, clrLine = clrLine, clrMark = clrMark)

ImgShow(imageSides, 200)


In [ ]:
# Find the play area corners

print(mksPaT)
ptTL = FindIntersection(mksPaL, mksPaT)
ptTR = FindIntersection(mksPaT, mksPaR)
ptBL = FindIntersection(mksPaL, mksPaB)
ptBR = FindIntersection(mksPaR, mksPaB)

imageCorners = imageOriginal.copy()

# Draw the new play area lines in a different color
clrLine = (255,255,0)
clrMark= (255,0,0)
DrawMarkers(imageCorners, mksPaT, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageCorners, mksPaB, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageCorners, mksPaL, clrLine = clrLine, clrMark = clrMark)
DrawMarkers(imageCorners, mksPaR, clrLine = clrLine, clrMark = clrMark)

print(ptTL)
dia = 175
clrCorner = (0,255,0)
fill = 10
cv.circle(imageCorners, ptTL, dia, clrCorner, fill)
cv.circle(imageCorners, ptTR, dia, clrCorner, fill)
cv.circle(imageCorners, ptBL, dia, clrCorner, fill)
cv.circle(imageCorners, ptBR, dia, clrCorner, fill)

ImgShow([imageCorners], 220)

In [ ]:
# Create a new transform with our new four corner points

# Build the array of src and dst points
ptsTransformSrc = np.array( [
    ToPtArray(ptTL),
    ToPtArray(ptTR), 
    ToPtArray(ptBR),
    ToPtArray(ptBL) ]
         ).astype(np.float32)

ptsTransformDst = np.array( [    
    [0,0],
    [tgtDims[0]-1, 0],
    [tgtDims[0]-1, tgtDims[1]-1],
    [0, tgtDims[1]-1]] 
         ).astype(np.float32)

# Calculate the transform matrix
matTxToFlat = cv.getPerspectiveTransform(ptsTransformSrc, ptsTransformDst)
matTxFromFlat = cv.getPerspectiveTransform(ptsTransformDst, ptsTransformSrc)

# Apply the warp transform
imageWarp = cv.warpPerspective(imageCorners, matTxToFlat, tgtDims)
print("imageWarp", imageWarp.shape)

print("Perspective Transform")
ImgShow([imageCorners, imageWarp],240)

In [ ]:
# Build a mask for the playable area
polyPlayable = np.array([ptTL, ptTR, ptBR, ptBL])
print("polyPlayable", polyPlayable)
clr = (255,255,255)
imageBlack = np.zeros(imageOriginal.shape[:2], np.uint8)
imageFeltMask = cv.fillPoly(imageBlack, [polyPlayable], color=clr)
imageFelt = cv.bitwise_and(imageOriginal, imageOriginal, mask=imageFeltMask)

ImgShow([imageOriginal, imageFeltMask, imageFelt],250)

In [ ]:
imageInput = imageFelt

imageHSV = cv.cvtColor(imageInput, cv.COLOR_BGR2HSV)
imageH = imageHSV[:, :, 0]
imageS = imageHSV[:, :, 1]
imageV = imageHSV[:, :, 2]

imageHSL = cv.cvtColor(imageInput, cv.COLOR_BGR2HLS)
imageH2 = imageHSV[:, :, 0]
imageS2 = imageHSV[:, :, 1]
imageL2 = imageHSV[:, :, 2]

imageG = cv.cvtColor(imageInput, cv.COLOR_BGR2GRAY)
ret, imageT = cv.threshold(imageG, 80, 255, 0)
ImgShow([imageH,imageS,imageV, imageT], 220)
ImgShow([imageH2,imageS2, imageL2, imageG], 220)

In [ ]:
# Play with histogram data to prepare for ball find
imageInput = imageFelt

imageHSV = cv.cvtColor(imageInput, cv.COLOR_BGR2HSV)
imagePlane = imageHSV[:, :, 2]

imagePreBalls = imagePlane
def OnGui(clipLimit, tileSize, thresh):
    print(f'clipLimit={clipLimit} tileSize={tileSize} thresh={thresh}')
    hist = cv.calcHist([imagePlane],[0],imageFeltMask,[256],[0,256])

    imagePlaneEq = cv.equalizeHist(imagePlane)
    clahe = cv.createCLAHE(clipLimit=clipLimit, tileGridSize=(tileSize,tileSize))
    imagePlaneClahe = clahe.apply(imagePlane)
    
    imgTMask = imagePlane > thresh            
    imageThresh = imgTMask * imagePlane

    ImgShow([imagePlane, imagePlaneEq, imageThresh, imagePlaneClahe], 220)
    global imagePreBalls
    imagePreBalls = imagePlaneEq
    
    
w = interactive(OnGui,  clipLimit=widgets.FloatSlider(min=0.0, max=50.0, value=1.0, step=0.1),
                        tileSize=widgets.IntSlider(min=1, max=500, value=8, step=2),
                        thresh=widgets.IntSlider(min=0, max=150, value=8, step=1)
                        )
display(w)


In [ ]:
# Find the balls using FindContours
imageInput = imagePlane

imageCanny = cv.Canny(imageInput, 100,200)
contours, hierarchy = cv.findContours(imageCanny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
#print(hierarchy)
imageBallContours = imageFelt.copy()
cv.drawContours(imageBallContours, contours, -1, (0,0,255), 3)

ImgShow([imageBallContours], 220)

In [ ]:
# Find the balls
from IPython.display import display
import ipywidgets as widgets

# Hough Circles. 
imageInput = imagePreBalls
print("imageInput.shape = {0}".format(imageInput.shape))

# Params for finding the circles
if True:
    mode = cv.HOUGH_GRADIENT
    blur = int(0)
    houghParam1=160
    houghParam2=25
    minDist=20
    minRadius=19
    maxRadius=40
else:
    mode = cv.HOUGH_GRADIENT_ALT
    blur = int(21)
    houghParam1=200
    houghParam2=0.01
    minDist=15
    
#imageGray = cv.cvtColor(imageInput, cv.COLOR_BGR2GRAY)

def FindBalls(image, bAltMode, blur, p1, p2, minDist, minRadius, maxRadius):
    
    if bAltMode:
        mode = cv.HOUGH_GRADIENT_ALT
    else:
        mode = cv.HOUGH_GRADIENT

    if blur > 0:
        imageBlur = cv.GaussianBlur(image, (blur,blur), cv.BORDER_DEFAULT)
    else:
        imageBlur = image
        
    hcircles = cv.HoughCircles(imageBlur, mode, minDist=minDist, dp=1,
                               param1=p1, param2=p2,
                               minRadius=minRadius, maxRadius=maxRadius)

    # Create an array of points
    balls = []
    imageBalls = imageOriginal.copy()
    if hcircles is not None:
        hcircles = np.uint16(np.around(hcircles))
        for i in hcircles[0, :]: 
            pt = [i[0], i[1]]
            rad = i[2]    
            balls.append(pt)
            cv.circle(imageBalls, pt, rad, Color(), 7)
    return balls, imageBalls


def OnGui(bAltMode, blur, p1, p2, minRadius, maxRadius):
    # Blur size must be odd
    blur2 = blur
    if 0 == (blur2 % 2):
        blur2 -= 1
    minDist = minRadius
    balls, imageBalls = FindBalls(imageInput, bAltMode, blur2, p1, p2, minDist, minRadius, maxRadius)
    ImgShow([imageInput, imageBalls],210)            
    print("balls count = {0}".format(len(balls)))
    
w = interactive(OnGui,  bAltMode = widgets.Checkbox(value = False),
                        blur=widgets.IntSlider(min=0, max=71, value=10, step=2),
                        p1=widgets.IntSlider(min=1, max=500, value=houghParam1, step=2),
                        p2=widgets.IntSlider(min=1, max=500, value=houghParam2, step=2),
                        minRadius=widgets.IntSlider(min=1, max=100, value=23, step=2),
                        maxRadius=widgets.IntSlider(min=1, max=400, value=40, step=2),
                       )
display(w)

In [ ]:
imagePlane = np.array([[0,1,2,3], [4,5,6,7]], np.int8)
thresh = 2
imgTMask = imagePlane > thresh            
imageThresh = imgTMask * imagePlane
print(imagePlane)
print(imgTMask)
print(imageThresh)